# 서울특별시 공공자전거 한강 근처 대여이력 분석

## 목표
- 서울특별시 공공자전거 대여이력 정보에서 한강 근처 대여소만 추출
- 한강 관련 키워드로 필터링하여 분석

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from matplotlib import rcParams
import warnings
import os

# 한글 폰트 경고 무시
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# Windows 시스템 폰트를 직접 로드하는 함수
def setup_korean_font():
    """Windows 시스템의 한글 폰트를 직접 로드하고 설정"""
    
    # 폰트 파일 경로들 (우선순위 순)
    font_paths = [
        r'C:\Windows\Fonts\malgun.ttf',      # 맑은 고딕
        r'C:\Windows\Fonts\malgunbd.ttf',    # 맑은 고딕 Bold
        r'C:\Windows\Fonts\gulim.ttc',       # 굴림
        r'C:\Windows\Fonts\batang.ttc'       # 바탕
    ]
    
    korean_font = None
    
    for font_path in font_paths:
        if os.path.exists(font_path):
            try:
                # 폰트를 직접 로드
                korean_font = fm.FontProperties(fname=font_path)
                
                # matplotlib 폰트 매니저에 폰트 추가
                fm.fontManager.addfont(font_path)
                
                print(f"✅ 폰트 로드 성공: {font_path}")
                print(f"   폰트명: {korean_font.get_name()}")
                
                # matplotlib 전역 설정에 폰트 적용
                font_name = korean_font.get_name()
                plt.rcParams['font.family'] = font_name
                rcParams['font.family'] = font_name
                plt.rcParams['axes.unicode_minus'] = False
                
                return korean_font
                
            except Exception as e:
                print(f"❌ {font_path} 로드 실패: {e}")
                continue
    
    print("❌ 모든 한글 폰트 로드 실패")
    return None

# 한글 폰트 설정 실행
korean_font_prop = setup_korean_font()

# seaborn 스타일 설정
sns.set_style("whitegrid")

print("라이브러리 로드 완료")


✅ 폰트 로드 성공: C:\Windows\Fonts\malgun.ttf
   폰트명: Malgun Gothic
라이브러리 로드 완료


In [2]:
# 데이터 파일 로드
file_path = '../서울특별시 공공자전거 대여이력 정보_2407.csv'

try:
    # CSV 파일 읽기 (utf-8 인코딩)
    df = pd.read_csv(file_path, encoding='utf-8')
    print(f"✅ 데이터 로드 성공: {len(df):,}행 {len(df.columns)}열")
    print(f"컬럼명: {list(df.columns)}")
    print(f"\n데이터 미리보기:")
    print(df.head())
    
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
    print("상위 디렉토리의 파일 목록:")
    import os
    for file in os.listdir('..'):
        if file.endswith('.csv'):
            print(f"  - {file}")
except Exception as e:
    print(f"❌ utf-8 인코딩으로 데이터 로드 실패: {e}")
    # 다른 인코딩들로 순차적으로 시도
    encodings_to_try = ['utf-8-sig', 'cp949', 'euc-kr', 'latin-1', 'cp1252']
    
    for encoding in encodings_to_try:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ {encoding} 인코딩으로 로드 성공: {len(df):,}행 {len(df.columns)}열")
            print(f"컬럼명: {list(df.columns)}")
            print(f"\n데이터 미리보기:")
            print(df.head())
            break
        except Exception as e2:
            print(f"❌ {encoding} 인코딩도 실패: {e2}")
            continue
    else:
        print("❌ 모든 인코딩 시도 실패")


❌ utf-8 인코딩으로 데이터 로드 실패: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
❌ utf-8-sig 인코딩도 실패: 'utf-8' codec can't decode byte 0xc0 in position 1: invalid start byte
✅ cp949 인코딩으로 로드 성공: 3,723,139행 17열
컬럼명: ['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납일시', '반납대여소번호', '반납대여소명', '반납거치대', '이용시간(분)', '이용거리(M)', '생년', '성별', '이용자종류', '대여대여소ID', '반납대여소ID', '자전거구분']

데이터 미리보기:
       자전거번호                 대여일시 대여 대여소번호                  대여 대여소명 대여거치대  \
0  SPB-47611  2024-07-01 00:00:17    01150                 송정역 1번출구     0   
1  SPB-39220  2024-07-01 00:00:29    05301  상계주공1단지 버스정류장 옆(대원빌딩 앞)     0   
2  SPB-34502  2024-07-01 00:00:44    04078                   원당공원 앞     0   
3  SPB-46189  2024-07-01 00:00:06    05052                 마곡역 7번출구     0   
4  SPB-53446  2024-07-01 00:00:19    00965               서울특별시 은평병원     0   

                  반납일시 반납대여소번호         반납대여소명 반납거치대  이용시간(분)  이용거리(M)    생년  \
0  2024-07-01 00:02:24   01109     공항시장역 4번출구     0     

In [3]:
# 한강 관련 키워드 정의
hangang_keywords = [
    '한강', '한강공원', '뚝섬', '여의도', '반포', '잠원', '이촌', '망원',
    '양화', '선유도', '잠실', '광나루', '구리', '강서', '난지', '원효대교',
    '한남대교', '동작대교', '반포대교', '잠수교', '한강대교', '마포대교',
    '서강대교', '양화대교', '성산대교', '가양대교', '올림픽대교', '천호대교',
    '영동대교', '성수대교', '잠실대교', '석촌호수', '뚝섬유원지', '여의나루',
    '당산철교', '방화대교', '행주대교'
]

print(f"한강 관련 키워드 {len(hangang_keywords)}개 정의 완료")
print("키워드 목록:", hangang_keywords)


한강 관련 키워드 37개 정의 완료
키워드 목록: ['한강', '한강공원', '뚝섬', '여의도', '반포', '잠원', '이촌', '망원', '양화', '선유도', '잠실', '광나루', '구리', '강서', '난지', '원효대교', '한남대교', '동작대교', '반포대교', '잠수교', '한강대교', '마포대교', '서강대교', '양화대교', '성산대교', '가양대교', '올림픽대교', '천호대교', '영동대교', '성수대교', '잠실대교', '석촌호수', '뚝섬유원지', '여의나루', '당산철교', '방화대교', '행주대교']


In [ ]:
# 한강 근처 대여소 필터링
if 'df' in locals():
    # 대여소명과 반납소명에서 한강 관련 키워드 검색
    hangang_mask = df.apply(lambda row: 
        any(keyword in str(row).lower() for keyword in [k.lower() for k in hangang_keywords] 
            for col in ['대여소', '대여소명', '반납소', '반납소명'] if col in df.columns),
        axis=1
    )
    
    # 더 정확한 필터링을 위해 각 컬럼별로 체크
    rental_cols = [col for col in ['대여소', '대여소명', '대여소_명'] if col in df.columns]
    return_cols = [col for col in ['반납소', '반납소명', '반납소_명'] if col in df.columns]
    
    hangang_condition = False
    
    # 대여소명에서 한강 키워드 검색
    for col in rental_cols:
        if col in df.columns:
            condition = df[col].astype(str).str.contains('|'.join(hangang_keywords), case=False, na=False)
            hangang_condition = hangang_condition | condition
            print(f"{col}에서 한강 관련: {condition.sum():,}건")
    
    # 반납소명에서 한강 키워드 검색
    for col in return_cols:
        if col in df.columns:
            condition = df[col].astype(str).str.contains('|'.join(hangang_keywords), case=False, na=False)
            hangang_condition = hangang_condition | condition
            print(f"{col}에서 한강 관련: {condition.sum():,}건")
    
    # 한강 관련 데이터 추출
    hangang_df = df[hangang_condition].copy()
    
    print(f"\n=== 한강 관련 대여이력 추출 결과 ===")
    print(f"전체 데이터: {len(df):,}건")
    print(f"한강 관련 데이터: {len(hangang_df):,}건")
    print(f"비율: {len(hangang_df)/len(df)*100:.1f}%")
    
    if len(hangang_df) > 0:
        print(f"\n한강 관련 데이터 미리보기:")
        print(hangang_df.head(10))
    else:
        print("❌ 한강 관련 데이터를 찾을 수 없습니다.")
        print("사용 가능한 컬럼명을 확인해주세요.")
        print("컬럼명:", list(df.columns))
else:
    print("데이터가 로드되지 않았습니다.")


In [ ]:
# 한강 관련 대여소 상위 분석
if 'hangang_df' in locals() and len(hangang_df) > 0:
    print("=== 한강 관련 주요 대여소 분석 ===")
    
    # 대여소별 이용 건수 분석
    rental_cols = [col for col in ['대여소', '대여소명', '대여소_명'] if col in hangang_df.columns]
    return_cols = [col for col in ['반납소', '반납소명', '반납소_명'] if col in hangang_df.columns]
    
    if rental_cols:
        rental_col = rental_cols[0]
        top_rental_stations = hangang_df[rental_col].value_counts().head(10)
        
        print(f"\n상위 10개 대여소 ({rental_col}):")
        for i, (station, count) in enumerate(top_rental_stations.items(), 1):
            print(f"{i:2d}. {station}: {count:,}건")
    
    if return_cols:
        return_col = return_cols[0]
        top_return_stations = hangang_df[return_col].value_counts().head(10)
        
        print(f"\n상위 10개 반납소 ({return_col}):")
        for i, (station, count) in enumerate(top_return_stations.items(), 1):
            print(f"{i:2d}. {station}: {count:,}건")
    
    # 시간대별 분석 (시간 컬럼이 있는 경우)
    time_cols = [col for col in ['대여시간', '대여일시', '이용시간', '시간'] if col in hangang_df.columns]
    if time_cols:
        print(f"\n시간 관련 컬럼 발견: {time_cols}")
        print("시간대별 분석이 가능합니다.")
    
else:
    print("한강 관련 데이터가 없습니다.")


In [ ]:
# 한강 관련 데이터 시각화
if 'hangang_df' in locals() and len(hangang_df) > 0 and 'top_rental_stations' in locals():
    # 상위 대여소 막대 그래프
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    top_10_rental = top_rental_stations.head(10)
    
    # 대여소명을 간략하게 줄이기
    short_names = [name[:30] + '...' if len(name) > 30 else name for name in top_10_rental.index]
    
    bars = plt.barh(range(len(top_10_rental)), top_10_rental.values, color='#2E86AB', alpha=0.8)
    plt.yticks(range(len(top_10_rental)), short_names, fontproperties=korean_font_prop, fontsize=9)
    plt.xlabel('이용 건수', fontproperties=korean_font_prop)
    plt.title('한강 관련 상위 10개 대여소 이용 현황 (2024년 7월)', 
             fontproperties=korean_font_prop, fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    
    # 수치 표시
    for i, (bar, value) in enumerate(zip(bars, top_10_rental.values)):
        plt.text(bar.get_width() + max(top_10_rental.values) * 0.02, 
                bar.get_y() + bar.get_height()/2, 
                f'{value:,}', ha='left', va='center', fontweight='bold', fontsize=8)
    
    # 반납소 그래프 (있는 경우)
    if 'top_return_stations' in locals():
        plt.subplot(2, 1, 2)
        top_10_return = top_return_stations.head(10)
        
        short_names_return = [name[:30] + '...' if len(name) > 30 else name for name in top_10_return.index]
        
        bars2 = plt.barh(range(len(top_10_return)), top_10_return.values, color='#A23B72', alpha=0.8)
        plt.yticks(range(len(top_10_return)), short_names_return, fontproperties=korean_font_prop, fontsize=9)
        plt.xlabel('이용 건수', fontproperties=korean_font_prop)
        plt.title('한강 관련 상위 10개 반납소 이용 현황 (2024년 7월)', 
                 fontproperties=korean_font_prop, fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        
        # 수치 표시
        for i, (bar, value) in enumerate(zip(bars2, top_10_return.values)):
            plt.text(bar.get_width() + max(top_10_return.values) * 0.02, 
                    bar.get_y() + bar.get_height()/2, 
                    f'{value:,}', ha='left', va='center', fontweight='bold', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n한강 관련 따릉이 이용 분석이 완료되었습니다.")
    print(f"총 {len(hangang_df):,}건의 한강 관련 이용 기록을 추출했습니다.")
else:
    print("시각화할 데이터가 없습니다.")
